In [ ]:
import boto3
from datetime import datetime

# Print execution info
print(f"Execution started at: {datetime.now()}")

# Get current IAM role
sts = boto3.client('sts')
identity = sts.get_caller_identity()
print(f"Current Role ARN: {identity['Arn']}")
print(f"Account ID: {identity['Account']}")

# ML Pipeline Orchestrator Notebook
This notebook orchestrates the complete ML pipeline using SageMaker SDK

In [ ]:
# Parameters cell - will be replaced by Papermill
mlflow_tracking_server_arn = None
mlflow_artifact_location = None  # Optional: Custom artifact location
sklearn_version = "1.2-1"
python_version = "py3"
training_instance_type = "ml.m5.large"
inference_instance_type = "ml.m5.large"
model_name = "realistic-classifier-v1"


In [ ]:
import os
print(f"IMAGE_VERSION: {os.environ.get('IMAGE_VERSION', 'Not set')}")
print(f"SAGEMAKER_INTERNAL_IMAGE_URI: {os.environ.get('SAGEMAKER_INTERNAL_IMAGE_URI', 'Not set')}")

In [ ]:
from datetime import datetime
from zoneinfo import ZoneInfo
print(f"Execution Start Time (EST): {datetime.now(ZoneInfo('America/New_York')).strftime('%Y-%m-%d %H:%M:%S %Z')}")


In [ ]:
# Package upgrade skipped - using environment default
print('Using default sagemaker_studio package from environment')


In [ ]:
import argparse
import os
import json
import time
import boto3
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import mlflow

print("🚀 Starting Complete ML Pipeline Orchestration with Real SageMaker SDK")

In [ ]:
from sagemaker_studio import Project

# Get project connections dynamically
# Get region from execution environment
region = boto3.Session().region_name

proj = Project()

# Get region from boto3 session
region = boto3.Session().region_name
print(f"✅ Region: {region}")

# Get S3 shared bucket from connection
s3_shared_conn = proj.connection('default.s3_shared')
bucket = s3_shared_conn.data.s3_uri.rstrip('/').split('/')[-2]  # Extract bucket name
print(f"✅ S3 Bucket from connection: {bucket}")

# Get IAM role from connection
iam_conn = proj.connection('default.iam')
role = iam_conn.iam_role
print(f"✅ IAM Role from connection: {role}")

# MLflow tracking server ARN
# MLflow tracking server ARN from parameter - must be provided
if not mlflow_tracking_server_arn:
    raise ValueError("mlflow_tracking_server_arn parameter is required but was not provided")

mlflow_arn = mlflow_tracking_server_arn
print(f"✅ MLflow ARN: {mlflow_arn}")


In [ ]:
# Generate synthetic training and inference data
print("\n📊 Generating synthetic training and inference data...")

import pandas as pd
import numpy as np

# Generate training data
np.random.seed(42)
X_train = np.random.randn(1000, 20)
y_train = np.random.choice([0, 1, 2], 1000)
train_df = pd.DataFrame(X_train, columns=[f'feature_{i}' for i in range(20)])
train_df['target'] = y_train

# Generate inference data (no target column)
np.random.seed(123)
X_test = np.random.randn(100, 20)
test_df = pd.DataFrame(X_test, columns=[f'feature_{i}' for i in range(20)])

# Upload to S3
s3_client = boto3.client('s3', region_name=region)

# Save and upload training data
train_local = '/tmp/training_data.csv'
train_df.to_csv(train_local, index=False)
train_s3_key = 'shared/ml/bundle/training-data/training_data.csv'
s3_client.upload_file(train_local, bucket, train_s3_key)
print(f"✅ Uploaded training data to s3://{bucket}/{train_s3_key}")

# Save and upload inference data
test_local = '/tmp/inference_data.csv'
test_df.to_csv(test_local, index=False)
test_s3_key = 'shared/ml/bundle/inference-data/inference_data.csv'
s3_client.upload_file(test_local, bucket, test_s3_key)
print(f"✅ Uploaded inference data to s3://{bucket}/{test_s3_key}")

In [ ]:
# Configuration
model_name = "realistic-classifier-v1"

# MLflow setup (only if available)
if mlflow_arn:
    mlflow.set_tracking_uri(mlflow_arn)
    print(f"✅ MLflow tracking enabled: {mlflow_arn}")
else:
    print("⚠️  MLflow tracking disabled (no connection)")

# Create SageMaker session
boto_session = boto3.Session(region_name=region)
session = sagemaker.Session(boto_session=boto_session)

print(f"\n📋 Configuration:")
print(f"  Region: {region}")
print(f"  Bucket: {bucket}")
print(f"  Role: {role}")
print(f"  Model: {model_name}")

In [ ]:
# Step 1: Training with Real SageMaker SDK
print("\n📈 Step 1: Real Model Training with SageMaker SDK")

# Build environment variables (only add MLflow if available)
env_vars = {}
if mlflow_arn:
    env_vars["MLFLOW_TRACKING_SERVER_ARN"] = mlflow_arn
    if mlflow_artifact_location:
        env_vars["MLFLOW_ARTIFACT_LOCATION"] = mlflow_artifact_location

source_dir = f's3://{bucket}/shared/ml/bundle/training-code/training-code.tar.gz'
training_data = f's3://{bucket}/shared/ml/bundle/training-data/'
output_path = f's3://{bucket}/shared/ml/output/model-artifacts/'

print(f"  Training code: {source_dir}")
print(f"  Training data: {training_data}")
print(f"  Output path: {output_path}")

sklearn_estimator = SKLearn(
    entry_point='sagemaker_training_script.py',
    source_dir=source_dir,
    framework_version=sklearn_version,
    py_version=python_version,
    instance_type=training_instance_type,
    instance_count=1,
    role=role,
    output_path=output_path,
    environment=env_vars,
    hyperparameters={
        'n-estimators': 100, 
        'max-depth': 10, 
        'random-state': 42,
        'model-name': model_name
    }
)

job_name = f"orchestrated-training-{int(time.time())}"
print(f"Starting real training job: {job_name}")

# This will create a real SageMaker training job
sklearn_estimator.fit(
    inputs={'training': training_data},
    job_name=job_name
)

print(f"✅ Real training completed: {sklearn_estimator.model_data}")

In [ ]:
# Copy model to 'latest' location for deployment
print("\n📋 Copying model to latest location...")

import boto3
s3_client = boto3.client('s3', region_name=region)

# Get model S3 path from training job
sm_client = boto3.client('sagemaker', region_name=region)
training_job_desc = sm_client.describe_training_job(TrainingJobName=job_name)
model_s3_uri = training_job_desc['ModelArtifacts']['S3ModelArtifacts']

# Parse S3 URI
source_bucket = model_s3_uri.split('/')[2]
source_key = '/'.join(model_s3_uri.split('/')[3:])
dest_key = 'shared/ml/output/model-artifacts/latest/output/model.tar.gz'

# Copy to latest
s3_client.copy_object(
    CopySource={'Bucket': source_bucket, 'Key': source_key},
    Bucket=bucket,
    Key=dest_key
)

latest_uri = f's3://{bucket}/{dest_key}'
print(f"   ✅ Model copied to: {latest_uri}")
print(f"   Original: {model_s3_uri}")

In [ ]:
# Step 2: Champion Model Tagging with Real MLflow
print("\n🏆 Step 2: Real Champion Model Tagging")
champion_tagged = False

if not mlflow_arn:
    print("⚠️  Skipping champion tagging (no MLflow connection)")
else:
    try:
        client = mlflow.MlflowClient()
        
        # Get latest model version
        versions = client.search_model_versions(f"name='{model_name}'")
        if versions:
            latest_version = max(versions, key=lambda x: int(x.version))
            client.set_registered_model_alias(
                name=model_name,
                alias="champion",
                version=latest_version.version
            )
            print(f"✅ Real champion tagging: version {latest_version.version}")
            champion_tagged = True
        else:
            print("⚠️  No model versions found, will be created by training job")
    except Exception as e:
        print(f"⚠️  Champion tagging will happen after training job completes: {e}")